In [9]:
!pip install tensorflow-tpu
!pip install git+https://github.com/openai/baselines.git@ea25b9e8b234e6ee1bca43083f8f3cf974143998

ERROR: Could not find a version that satisfies the requirement tensorflow-tpu (from versions: none)
ERROR: No matching distribution found for tensorflow-tpu
  Cloning https://github.com/openai/baselines.git (to revision ea25b9e8b234e6ee1bca43083f8f3cf974143998) to /tmp/pip-req-build-e5d3poz5
  Running command git clone -q https://github.com/openai/baselines.git /tmp/pip-req-build-e5d3poz5
  Created wheel for baselines: filename=baselines-0.1.6-cp37-none-any.whl size=220681 sha256=d1d70aadb0891e0b243724f2538a28c2e56e188dbb3d62b94be10d1dd5ec32ad
  Stored in directory: /tmp/pip-ephem-wheel-cache-m14i8t8l/wheels/d1/e9/68/924a24fbf16f3feb881d97210a6379eeb41c0fd6f930112a39
Successfully built baselines


In [10]:
%tensorflow_version 1.x
import time
import gym
import random
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import os

from baselines.ppo2 import ppo2
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv

#from stable_baselines import DQN
from baselines import deepq
#from stable_baselines import deepq
from baselines import bench
from baselines import logger
import tensorflow as tf

from baselines.common.tf_util import make_session

In [12]:
num_classes = 10
input_shape = (32, 32, 3)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train_one_hot = keras.utils.to_categorical(y_train, num_classes)
y_test_one_hot = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3, 1)
50000 train samples
10000 test samples


In [13]:
class cifar10Env(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        super().__init__()

        self.action_space = gym.spaces.Discrete(10)
        self.observation_space = gym.spaces.Box(low=0, high=1,
                                                shape=(32, 32, 3),
                                                dtype=np.float32)

        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0

    def step(self, action):
        done = False
        reward = int(action == self.expected_action)

        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {}

    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y[next_obs_idx])
            obs = self.x[next_obs_idx]

        else:
            obs = self.x[self.dataset_idx]
            self.expected_action = int(self.y[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()

        return obs



In [ ]:
def cifar10_dqn():
    logger.configure(dir='./logs/cifar10_dqn', format_strs=['stdout', 'tensorboard'])
    env = cifar10Env(images_per_episode=1)
    env = bench.Monitor(env, logger.get_dir())

    model = deepq.learn(
        env,
        "mlp",
        num_layers=1,
        num_hidden=128,
        activation=tf.nn.relu,
        hiddens=[32],
        dueling=True,
        lr=1e-4,
        total_timesteps=int(1.2e6),
        buffer_size=100000,
        exploration_fraction=0.1,
        exploration_final_eps=0.01,
        train_freq=4,
        learning_starts=10000,
        target_network_update_freq=10000,
    )

    model.save('dqn_cifar10.pkl')
    env.close()
    
    return model

start_time = time.time()
dqn_model = cifar10_dqn()
print("DQN Training Time:", time.time() - start_time)

Logging to ./logs/cifar10_dqn





/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Streaming output truncated to the last 5000 lines.
--------------------------------------
| % time spent exploring  | 1        |
| episodes                | 1.12e+06 |
| mean 100 episode reward | 0.6      |
| steps                   | 1.12e+06 |
--------------------------------------
--------------------------------------
| % time spent exploring  | 1        |
| episodes                | 1.12e+06 |
| mean 100 episode reward | 0.6      |
| steps                   | 1.12e+06 |
--------------------------------------
--------------------------------------
| % time spent exploring  | 1        |
| episodes                | 1.12e+06 |
| mean 100 episode reward | 0.5      |
| steps                   | 1.12e+06 |
--------------------------------------
--------------------------------------
| % time spent exploring  | 1        |
| episodes                | 1.12e+06 |
| mean 100 episode reward | 0.6      |
| steps                   | 1.12e+06 |
--------------------------------------
-------------

In [ ]:
def cifar10_dqn_eval(dqn_model):
    attempts, correct = 0,0

    env = cifar10Env(images_per_episode=1, dataset=(x_test, y_test), random=False)
    
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, _ = env.step(dqn_model(obs[None])[0])

                attempts += 1
                if rew > 0:
                    correct += 1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))

cifar10_dqn_eval(dqn_model)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 44.56891378275655%


In [14]:
def cifar10_ppo():
    logger.configure(dir='./logs/cifar10_ppo', format_strs=['stdout', 'tensorboard'])
    env = DummyVecEnv([lambda: bench.Monitor(cifar10Env(images_per_episode=1), logger.get_dir())])

    model = ppo2.learn(
        env=env,
        network='mlp',
        num_layers=2,
        num_hidden=64,
        nsteps=32,
        total_timesteps=int(1.2e5),
        seed=int(time.time()))
    
    return model

start_time = time.time()
ppo_model = cifar10_ppo()
print("PPO Training Time:", time.time() - start_time)

Logging to ./logs/cifar10_ppo


/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


ValueError: ignored

In [ ]:
def cifar10_ppo_eval(ppo_model):
    attempts, correct = 0,0

    env = DummyVecEnv([lambda: cifar10Env(images_per_episode=1, dataset=(x_test, y_test), random=False)])
    
    try:
        while True:
            obs, done = env.reset(), [False]
            while not done[0]:
                obs, rew, done, _ = env.step(ppo_model.step(obs[None])[0])

                attempts += 1
                if rew[0] > 0:
                    correct += 1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))

cifar10_ppo_eval(ppo_model)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-c05e3343153f>", line 21, in <module>
    cifar10_ppo_eval(ppo_model)
NameError: name 'ppo_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ul

NameError: ignored